In [1]:
!(apt install -y fluidsynth lilypond && \
pip install --upgrade pyfluidsynth pretty_midi \
)&> /dev/null

In [2]:
import random
import numpy as np
import fluidsynth
import glob
import pathlib
import pretty_midi
import tensorflow as tf
from tensorflow import keras
from IPython import display
import music21
from music21 import midi
import torch
import torchaudio

In [3]:
data_file = keras.utils.get_file(
    'maestro-v3.0.0-midi.zip',
    origin='https://storage.googleapis.com/magentadata/datasets/maestro/v3.0.0/maestro-v3.0.0-midi.zip',
    extract=True, cache_dir='.'
)
print(data_file)
data_dir = pathlib.Path('datasets/maestro-v3.0.0')

58416533/58416533 [==============================] - 3s 0us/step
./datasets/maestro-v3.0.0-midi.zip


In [4]:
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

Number of files: 1276


In [5]:
sampling_rate = 32000

In [6]:
def play(pm, seconds=30):
  waveform = pm.fluidsynth(fs=sampling_rate)
  return display.Audio(waveform[:seconds*sampling_rate], rate=sampling_rate)

In [38]:
sample_file = random.choice(filenames)
print(sample_file)
pm = pretty_midi.PrettyMIDI(sample_file)

datasets/maestro-v3.0.0/2017/MIDI-Unprocessed_041_PIANO041_MID--AUDIO-split_07-06-17_Piano-e_1-01_wav--1.midi


In [39]:
play(pm)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
def pm_to_tensor(pm, window_size=50):
  piano_roll = pm.get_piano_roll()
  num_windows = piano_roll.shape[1] // window_size
  piano_roll = piano_roll[:, :num_windows * window_size]
  piano_roll = piano_roll.reshape(piano_roll.shape[0], -1, window_size)
  return torch.tensor(piano_roll, dtype=torch.float32)

In [35]:
def tensor_to_pm(piano_roll_tensor, program=0):
  piano_roll = piano_roll_tensor.numpy()
  piano_roll = piano_roll.reshape(piano_roll.shape[0], -1)
  return piano_roll_to_pretty_midi(piano_roll, program=program)

def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [40]:
sample_tensor = pm_to_tensor(pm)

In [41]:
pm = tensor_to_pm(sample_tensor)

In [42]:
play(pm)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print('Number of instruments:', len(pm.instruments))
print('Instruments:')
for instrument in pm.instruments:
  print(' *', pretty_midi.program_to_instrument_name(instrument.program))

Number of instruments: 1
Instruments:
 * Acoustic Grand Piano


In [ ]:
mf = midi.MidiFile()
mf.open(sample_file)
mf.read()
s = midi.translate.midiFileToStream(mf)
play(s.write('mid'))